In [33]:
import robosandbox as rsb
import numpy as np
import swift
import os


## Commercial models

## Read from URDF File

In [37]:
# get the current working directory
pwd = os.getcwd()
tld = os.path.join(pwd, "data", "SO101")

SO101 = rsb.models.URDF.Model(
    name="SO101",
    file_path="so101_new_calib.urdf",
    tld=tld,
)
SO101.q = np.zeros(6)
print(SO101)
SO101.plot(q=SO101.q)

ERobot: so101_new_calib, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬───────────┬───────┬───────────┬───────────────────────────────────────────────────────────────────────┐
│ link │   link    │ joint │  parent   │                          ETS: parent to link                          │
├──────┼───────────┼───────┼───────────┼───────────────────────────────────────────────────────────────────────┤
│    0 │ base      │       │ BASE      │ SE3()                                                                 │
│    1 │ shoulder  │     0 │ base      │ SE3(0.02079, -0.02307, 0.09488; -180°, 3.459e-14°, 90°) ⊕ Rz(q0)      │
│    2 │ upper_arm │     1 │ shoulder  │ SE3(-0.0304, -0.01828, -0.0542; 90°, -90°, 180°) ⊕ Rz(q1)             │
│    3 │ lower_arm │     2 │ upper_arm │ SE3(-0.1126, -0.028, 2.463e-16; -7.037e-14°, 3.3e-14°, 90°) ⊕ Rz(q2)  │
│    4 │ wrist     │     3 │ lower_arm │ SE3(-0.1349, 0.0052, 1.652e-16; 1.861e-13°, 1.64e-13°, -90°) ⊕ Rz(q3) │
│    5 │ gripper   │  

Swift backend, t = 0.05, scene:
  so101_new_calib

In [ ]:
env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(SO101)
# env.close()

TypeError: teach() not supported for Swift backend